In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("kazanova/sentiment140")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/sentiment140


In [ ]:
df = pd.read_csv(path + "/training.1600000.processed.noemoticon.csv", encoding="latin-1", names=['sentiment', 'id', 'date', 'flag', 'user', 'text'])
df

,sentiment,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
...,...,...,...,...,...,...
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


In [ ]:
df = df[['text', 'sentiment']]  # Keep only text and label
df['sentiment'] = df['sentiment'].map({4: 1, 0: 0})
df

<ipython-input-6-ab70a5804ce4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sentiment'] = df['sentiment'].map({4: 1, 0: 0})


,text,sentiment
0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",0
1,is upset that he can't update his Facebook by ...,0
2,@Kenichan I dived many times for the ball. Man...,0
3,my whole body feels itchy and like its on fire,0
4,"@nationwideclass no, it's not behaving at all....",0
...,...,...
1599995,Just woke up. Having no school is the best fee...,1
1599996,TheWDB.com - Very cool to hear old Walt interv...,1
1599997,Are you ready for your MoJo Makeover? Ask me f...,1
1599998,Happy 38th Birthday to my boo of alll time!!! ...,1


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 2 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   text       1600000 non-null  object
 1   sentiment  1600000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 24.4+ MB


In [ ]:
df['sentiment'].value_counts()

,count
sentiment,
0,800000
1,800000


In [ ]:
!pip install nltk
import nltk
nltk.download('punkt_tab')
nltk.download('stopwords')
from nltk import word_tokenize, MWETokenizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
print(df['sentiment'].isnull().sum())
print(df['text'].isnull().sum())

0
0


In [ ]:
import string
!pip install contractions
from contractions import fix

In [ ]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)  # remove numbers
    text = re.sub(r'[^\w\s]', '', text)  # remove punctuation
    text = re.sub(r'\s+', ' ', text).strip()  # remove extra spaces


    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

    # Remove mentions (@user) and hashtags (#tag)
    text = re.sub(r'@\w+|#\w+', '', text)

     # Remove emojis (Unicode ranges)
    emoji_pattern = re.compile(
        "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        "]+", flags=re.UNICODE
    )
    text = emoji_pattern.sub(r'', text)

    text = fix(text)

    return text

df['text'] = df['text'].apply(clean_text)
df

<ipython-input-12-671e8da11c16>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].apply(clean_text)


,text,sentiment
0,switchfoot awww that is a bummer you shoulda ...,0
1,is upset that he cannot update his facebook by...,0
2,kenichan i dived many times for the ball manag...,0
3,my whole body feels itchy and like its on fire,0
4,nationwideclass no its not behaving at all i a...,0
...,...,...
1599995,just woke up having no school is the best feel...,1
1599996,thewdbcom very cool to hear old walt interview...,1
1599997,are you ready for your mojo makeover ask me fo...,1
1599998,happy th birthday to my boo of alll time tupac...,1


In [ ]:
from sklearn.model_selection import train_test_split

X = df['text']
y = df['sentiment']

# 80-20 split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
!pip uninstall gensim -y
!pip install gensim
import os
os.kill(os.getpid(), 9)

Found existing installation: gensim 4.3.3
Uninstalling gensim-4.3.3:
  Successfully uninstalled gensim-4.3.3
  Using cached gensim-4.3.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (8.1 kB)
Using cached gensim-4.3.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (26.7 MB)


In [ ]:
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
# Tokenize sentences
train_sentences = [word_tokenize(sent) for sent in X_train]

# Skip-gram model (better for rare words)
model = Word2Vec(
    sentences=train_sentences,
    vector_size=100,  # Embedding dimension
    window=5,        # Context window
    min_count=5,     # Ignore rare words
    sg=1,            # 1=Skip-gram, 0=CBOW
    workers=4        # Parallel threads
)

# Save model
model.save("twitter_word2vec.model")

# Example: Find similar words
print(model.wv.most_similar("happy", topn=5))

[('happpy', 0.7784025073051453), ('happppy', 0.7201628088951111), ('xsavedgex', 0.7095094323158264), ('dayhappy', 0.7008549571037292), ('katiesantry', 0.6943616271018982)]


In [ ]:
!pip install contractions tensorflow
!pip install tensorflow
!pip install keras

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences
import numpy as np

# Tokenize text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

# Convert text to sequences
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Pad sequences
maxlen = 100  # you can tune this
X_train_pad = pad_sequences(X_train_seq, maxlen=maxlen)
X_test_pad = pad_sequences(X_test_seq, maxlen=maxlen)

# Prepare embedding matrix from your Word2Vec model
word_index = tokenizer.word_index
embedding_dim = 100
embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))

for word, i in word_index.items():
    if word in model.wv:
        embedding_matrix[i] = model.wv[word]


In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.optimizers import Adam

# Build LSTM model
model_lstm = Sequential()

# Embedding layer using pre-trained Word2Vec weights
model_lstm.add(Embedding(input_dim=embedding_matrix.shape[0],
                         output_dim=embedding_matrix.shape[1],
                         weights=[embedding_matrix],
                         input_length=X_train_pad.shape[1],
                         trainable=False))  # set to True if you want to fine-tune

# LSTM layer
model_lstm.add(LSTM(units=128, return_sequences=False))

# Dropout to prevent overfitting
model_lstm.add(Dropout(0.5))

# Output layer
model_lstm.add(Dense(1, activation='sigmoid'))

# Compile the model
model_lstm.compile(loss='binary_crossentropy',
                   optimizer=Adam(learning_rate=0.001),
                   metrics=['accuracy'])

# Train the model
history = model_lstm.fit(X_train_pad, y_train,
                         validation_data=(X_test_pad, y_test),
                         epochs=5,
                         batch_size=128,
                         verbose=1)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5
10000/10000 ━━━━━━━━━━━━━━━━━━━━ 105s 10ms/step - accuracy: 0.7814 - loss: 0.4606 - val_accuracy: 0.8133 - val_loss: 0.4072
Epoch 2/5
10000/10000 ━━━━━━━━━━━━━━━━━━━━ 138s 10ms/step - accuracy: 0.8145 - loss: 0.4054 - val_accuracy: 0.8218 - val_loss: 0.3913
Epoch 3/5
10000/10000 ━━━━━━━━━━━━━━━━━━━━ 100s 10ms/step - accuracy: 0.8228 - loss: 0.3911 - val_accuracy: 0.8250 - val_loss: 0.3849
Epoch 4/5
10000/10000 ━━━━━━━━━━━━━━━━━━━━ 152s 11ms/step - accuracy: 0.8287 - loss: 0.3807 - val_accuracy: 0.8279 - val_loss: 0.3786
Epoch 5/5
10000/10000 ━━━━━━━━━━━━━━━━━━━━ 143s 11ms/step - accuracy: 0.8321 - loss: 0.3744 - val_accuracy: 0.8291 - val_loss: 0.3789


In [ ]:
from sklearn.metrics import classification_report

y_pred = (model_lstm.predict(X_test_pad) > 0.5).astype("int32")

print(classification_report(y_test, y_pred))


10000/10000 ━━━━━━━━━━━━━━━━━━━━ 30s 3ms/step
              precision    recall  f1-score   support

           0       0.82      0.84      0.83    159494
           1       0.84      0.82      0.83    160506

    accuracy                           0.83    320000
   macro avg       0.83      0.83      0.83    320000
weighted avg       0.83      0.83      0.83    320000

